<img src="../images/magic8ball.jpeg"  style="float: left; margin: 20px; height: 100px">

# Magic 8 balls

## Data Gathering
---

#### Library imports

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from sklearn.metrics import mean_squared_error

#### Total covid cases for all 50 states

Using the requests python Library, we fetched the total number of reported covid cases in all states from the first reported case, in Jan 23, 2020 to Dec 29, 2021.

In [47]:
#list of 51 US states 
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [48]:

#create empty dataframe 
df = pd.DataFrame(columns = ['submission_date','state','tot_cases'])

for state in states:
    url = 'https://data.cdc.gov/resource/9mfq-cb36.json?$select=submission_date,state,tot_cases&$where=submission_date%3E=%222020-01-23%22AND%20submission_date%3C%222021-12-30%22AND%20state=%27' + state + '%27&$limit=105000&$offset=0'
    res = requests.get(url)
    data = res.json()
    
    #convert to dataframe
    data = pd.DataFrame(data)
    
    #concat new dataframe with empty dataframe initialized above
    df = pd.concat([df,data])
    
    #so not to get ban from the internet
    time.sleep(3)

In [49]:
df.head()

,submission_date,state,tot_cases
0,2021-07-27T00:00:00.000,AK,71521
1,2020-04-16T00:00:00.000,AK,300
2,2021-02-25T00:00:00.000,AK,55950
3,2021-04-18T00:00:00.000,AK,63580
4,2021-11-05T00:00:00.000,AK,136822


#### Export dataframe for all the states

We exported the raw dataframe so we won't need to make repeated API requests

In [50]:
df.to_csv('../data/afolabi/all_states_raw.csv', index=False)

#### Re-import the csv

In [51]:
df = pd.read_csv('../data/afolabi/all_states_raw.csv')
df.head(2)

,submission_date,state,tot_cases
0,2021-07-27T00:00:00.000,AK,71521
1,2020-04-16T00:00:00.000,AK,300


#### Change the date column to datetime format and make index

- Using the date time library, convert the submission_date column to date time format
- Rename submisison_date as day
- Convert the tot_cases from object type to float

In [52]:
# From David's notebook

df['submission_date'] = pd.to_datetime(df['submission_date'])
df['tot_cases'] = df['tot_cases'].astype(float)

#create a data frame for infections
#create an index: 
days = (df[df['state'] == 'AK']).sort_values(by='submission_date')['submission_date'].values

In [53]:
df.head(2)

,submission_date,state,tot_cases
0,2021-07-27,AK,71521.0
1,2020-04-16,AK,300.0


Create a new dataframe called infected and set days as index

In [54]:
infected = pd.DataFrame(index = days)

For easy use in Tableau, set each state as it's own row

In [55]:
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
#            'HI', 
          'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
for state in states:
    #print(f'trying {state}')
    infected[f'{state}_I'] =  list((df[df['state'] == state]).sort_values(by='submission_date')['tot_cases'].diff(14))
    infected[f'{state}_I'][0:14] = infected[f'{state}_I'][15].copy()
infected.head(2)

,AK_I,AL_I,AR_I,AZ_I,CA_I,CO_I,CT_I,DC_I,DE_I,FL_I,...,SD_I,TN_I,TX_I,UT_I,VA_I,VT_I,WA_I,WI_I,WV_I,WY_I
2020-01-23,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
infected.head(2)

,AK_I,AL_I,AR_I,AZ_I,CA_I,CO_I,CT_I,DC_I,DE_I,FL_I,...,SD_I,TN_I,TX_I,UT_I,VA_I,VT_I,WA_I,WI_I,WV_I,WY_I
2020-01-23,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Export infected dataframe as csv to data folder

In [57]:
infected.to_csv('../data/infected_all_states.csv')

#### Total vaccinated 

Using the requests API, get the the total number of vaccinated people in all states

In [58]:
# from David

# initialize a df for vax
vax = pd.DataFrame()

states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
#            'HI', 
          'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
for state in states:
    time.sleep(1)
    url_vax = f'https://data.cdc.gov/resource/8xkx-amqh.json?$select=Date,Recip_State,Recip_County,Series_Complete_Yes&$where=Date%3E=%222020-12-01%22AND%20Date%3C%222022-03-21%22AND%20Recip_State=%27{state}%27&$limit=105000&$offset=0'
    #print(f'trying {state}')
    res_vax = requests.get(url_vax)
    if res_vax.status_code !=200:
        print(f'Status not ok for {state}')
        break
    data_vax= res_vax.json()
    df = pd.DataFrame(data_vax)
    # Make the data numerical and date time
    df['Date'] = pd.to_datetime(df['Date'])
    df['Series_Complete_Yes'] = df['Series_Complete_Yes'].astype(float)

    # sum over counties to get state level data
    df = df.groupby('Date').sum().sort_index()

    # write to the vax df
    vax[f'{state}_V'] = df['Series_Complete_Yes'].diff()
    vax[f'{state}_V'][0] = 0

    del df['Series_Complete_Yes']


In [59]:
vax.head(2)

,AK_V,AL_V,AR_V,AZ_V,CA_V,CO_V,CT_V,DC_V,DE_V,FL_V,...,SD_V,TN_V,TX_V,UT_V,VA_V,VT_V,WA_V,WI_V,WV_V,WY_V
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Merging infected dataframe and vaccinated dataframe

In [60]:
vax = pd.read_csv('../data/all_state_vax.csv')
vax['Date'] = pd.to_datetime(vax['Date'])
vax.set_index('Date',inplace=True)

In [61]:
vax.head(2)

,AK_V,AL_V,AR_V,AZ_V,CA_V,CO_V,CT_V,DC_V,DE_V,FL_V,...,SD_V,TN_V,TX_V,UT_V,VA_V,VT_V,WA_V,WI_V,WV_V,WY_V
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
infected_vaxed = pd.merge(infected, vax, 
         left_on= infected.index, 
         right_on= vax.index, 
         how='inner').set_index('key_0').fillna(0)


#### Export the merged infeced and vaccinated files

In [ ]:
infected_vaxed.to_csv('../data/I_V_all_states.csv')